# Initialize environment

In [11]:
# !pip install --upgrade pip && pip install "numpy>=2.1,<3" "crewai>=0.51.0" "crewai_tools>=0.1.6" "langchain-community>=0.2.10"

# Warning control
import warnings
from crewai import Agent, Task, Crew
import os
from IPython.display import Markdown
from crewai_tools import ScrapeWebsiteTool

warnings.filterwarnings('ignore')

ENV_OPENAI_API_KEY = "OPENAI_API_KEY"

def get_openai_api_key() -> str:
    """Return the OpenAI API key from the environment or raise with guidance."""
    api_key = os.getenv(ENV_OPENAI_API_KEY)
    if not api_key:
        raise RuntimeError(
            f"Missing {ENV_OPENAI_API_KEY}. Set it in your environment, e.g.,\n"
            f"export {ENV_OPENAI_API_KEY}='YOUR_KEY_HERE'"
        )
    return api_key

# Initialize environment and agent
openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

# Define Agents

In [2]:
def create_support_agent(*, allow_delegation: bool = False, verbose: bool = True) -> Agent:
    """Create and configure the Support agent."""
    return Agent(
        role="Senior Support Representative",
        goal="Be the most friendly and helpful "
            "support representative in your team",
        backstory=(
            "You work at crewAI (https://crewai.com) and "
            "are now working on providing "
            "support to {customer}, a super important customer "
            "for your company. "
            "You need to make sure that you provide the best support! "
            "Make sure to provide full complete answers, "
            "and make no assumptions."
        ),
        allow_delegation=allow_delegation,
        verbose=verbose,
    )

def create_quality_assurance_agent(*, allow_delegation: bool = True, verbose: bool = True) -> Agent:
    """Create and configure the Quality Assurance agent."""
    return Agent(
        role="Support Quality Assurance Specialist",
        goal="Get recognition for providing the "
        "best support quality assurance in your team",
        backstory=(
            "You work at crewAI (https://crewai.com) and "
            "are now working with your team "
            "on a request from {customer} ensuring that "
            "the support representative is "
            "providing the best support possible.\n"
            "You need to make sure that the support representative "
            "is providing full"
            "complete answers, and make no assumptions."
        ),
        allow_delegation=allow_delegation,
        verbose=verbose,
    )

# Create agent instances

In [3]:
support_agent = create_support_agent()
# Important to set allow_delegation to True for QA agent.
qa_agent = create_quality_assurance_agent(allow_delegation=True)

# Define tools

In [4]:
def create_scrape_tool(*, url: str) -> ScrapeWebsiteTool:
    """Create and configure the Plan task."""
    return ScrapeWebsiteTool(website_url=url)

# Define tasks

In [5]:
def create_inquiry_resolution_task(*, agent: Agent, tools) -> Task:
    """Create and configure the Inquiry Resolution task."""
    return Task(
        description=(
            "{customer} just reached out with a super important ask:\n"
            "{inquiry}\n\n"
            "{person} from {customer} is the one that reached out. "
            "Make sure to use everything you know "
            "to provide the best support possible."
            "You must strive to provide a complete "
            "and accurate response to the customer's inquiry."
        ),
        expected_output=(
            "A detailed, informative response to the "
            "customer's inquiry that addresses "
            "all aspects of their question.\n"
            "The response should include references "
            "to everything you used to find the answer, "
            "including external data or solutions. "
            "Ensure the answer is complete, "
            "leaving no questions unanswered, and maintain a helpful and friendly "
            "tone throughout."
        ),
        tools=tools,
        agent=agent,
    )

def create_quality_assurance_review_task(*, agent: Agent) -> Task:
    """Create and configure the Quality Assurance Review task."""
    return Task(
        description=(
            "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
            "Ensure that the answer is comprehensive, accurate, and adheres to the "
            "high-quality standards expected for customer support.\n"
            "Verify that all parts of the customer's inquiry "
            "have been addressed "
            "thoroughly, with a helpful and friendly tone.\n"
            "Check for references and sources used to "
            " find the information, "
            "ensuring the response is well-supported and "
            "leaves no questions unanswered."
        ),
        expected_output=(
            "A final, detailed, and informative response "
            "ready to be sent to the customer.\n"
            "This response should fully address the "
            "customer's inquiry, incorporating all "
            "relevant feedback and improvements.\n"
            "Don't be too formal, we are a chill and cool company "
            "but maintain a professional and friendly tone throughout."
        ),
        agent=agent,
    )

# Initiate tools

In [6]:
scrape_tool = create_scrape_tool(url='https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/')

# Create task instances

In [7]:
inquiry_resolution = create_inquiry_resolution_task(agent=support_agent, tools=[scrape_tool])
quality_assurance_review = create_quality_assurance_review_task(agent=qa_agent)

# Create crew

In [8]:
crew = Crew(
  agents=[support_agent, qa_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=True,
  memory=True
)

# Start crew

In [9]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}

result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 635b0e55-7c78-4f15-84e9-c48162e27c9d                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Support Representative                                                                           │
│                                                                                                                 │
│  Task: DeepLearningAI just reached out with a super important ask:                                              │
│  I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you   │
│  provide guidance?                                                                                              │
│                                                                                                                 │
│  Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide     │
│  the best support possible.You must strive to provide a complete and accurate response to the customer's        │
│  inquiry.                                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

[EventBus Error] Handler 'on_tool_usage_finished' failed for event 'ToolUsageFinishedEvent': 'Read website content'

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Support Quality Assurance Specialist                                                                    │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Dear DeepLearningAI team,                                                                                      │
│                                                                                                                 │
│  Thank you for reaching out to us at CrewAI with your inquiry about adding memory to your Crew using our        │
│  platform. We understand the importance of effectively managing memory within your Crew to enhance              │
│  collaboration and task execution.                                                                              │
│                                                                                                                 │
│  To add memory to your Crew in CrewAI, you can leverage the functionality of tasks and processes within a       │
│  Crew. Our framework enables you to define sequential or parallel workflows where tasks contribute to larger    │
│  processes, generating actionable outcomes. Each task in CrewAI has distinct objectives and can utilize         │
│  specific tools, allowing for the enhancement of the Crew's memory.                                             │
│                                                                                                                 │
│  By structuring tasks effectively and ensuring that the output of each task is stored or shared appropriately   │
│  within the Crew, you can create a cohesive memory system. This shared memory can be accessed by different      │
│  agents in the Crew as they collaborate on complex tasks, fostering efficiency and knowledge sharing.           │
│                                                                                                                 │
│  If you require further assistance on implementing memory management within your Crew using CrewAI, please      │
│  provide us with more details on your specific requirements. We are here to offer tailored guidance to help     │
│  you successfully establish and launch your Crew with an efficient memory management system.                    │
│                                                                                                                 │
│  For additional information and detailed instructions on creating a Crew and initiating tasks, you can refer    │
│  to our CrewAI Documentation at https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/.                │
│                                                                                                                 │
│  We are confident that with the right approach to memory management, your Crew will excel in handling tasks     │
│  and processes seamlessly. Should you have any more questions or need further assistance, feel free to reach    │
│  out to us anytime.                                                                                             │
│                                                                                                                 │
│  Best regards,                                                                                                  │
│  The CrewAI Support Team                                                                                        │
│                                                                                                                 │
╰────────────────────────────────────────────────────────

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: f4f48cff-cd2e-4b3c-b0e4-c4d1b8ecff80                                                                     │
│  Agent: Support Quality Assurance Specialist                                                                    │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 635b0e55-7c78-4f15-84e9-c48162e27c9d                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Dear DeepLearningAI team,                                                                        │
│                                                                                                                 │
│  Thank you for reaching out to us at CrewAI with your inquiry about adding memory to your Crew using our        │
│  platform. We understand the importance of effectively managing memory within your Crew to enhance              │
│  collaboration and task execution.                                                                              │
│                                                                                                                 │
│  To add memory to your Crew in CrewAI, you can leverage the functionality of tasks and processes within a       │
│  Crew. Our framework enables you to define sequential or parallel workflows where tasks contribute to larger    │
│  processes, generating actionable outcomes. Each task in CrewAI has distinct objectives and can utilize         │
│  specific tools, allowing for the enhancement of the Crew's memory.                                             │
│                                                                                                                 │
│  By structuring tasks effectively and ensuring that the output of each task is stored or shared appropriately   │
│  within the Crew, you can create a cohesive memory system. This shared memory can be accessed by different      │
│  agents in the Crew as they collaborate on complex tasks, fostering efficiency and knowledge sharing.           │
│                                                                                                                 │
│  If you require further assistance on implementing memory management within your Crew using CrewAI, please      │
│  provide us with more details on your specific requirements. We are here to offer tailored guidance to help     │
│  you successfully establish and launch your Crew with an efficient memory management system.                    │
│                                                                                                                 │
│  For additional information and detailed instructions on creating a Crew and initiating tasks, you can refer    │
│  to our CrewAI Documentation at https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/.                │
│                                                                                                                 │
│  We are confident that with the right approach to memory management, your Crew will excel in handling tasks     │
│  and processes seamlessly. Should you have any more questions or need further assistance, feel free to reach    │
│  out to us anytime.                                                                                             │
│                                                                                                                 │
│  Best regards,                                                                                                  │
│  The CrewAI Support Team                                                                                        │
│                                                       

# Results

In [10]:
Markdown(getattr(result, "raw", str(result)))

Dear DeepLearningAI team,

Thank you for reaching out to us at CrewAI with your inquiry about adding memory to your Crew using our platform. We understand the importance of effectively managing memory within your Crew to enhance collaboration and task execution.

To add memory to your Crew in CrewAI, you can leverage the functionality of tasks and processes within a Crew. Our framework enables you to define sequential or parallel workflows where tasks contribute to larger processes, generating actionable outcomes. Each task in CrewAI has distinct objectives and can utilize specific tools, allowing for the enhancement of the Crew's memory.

By structuring tasks effectively and ensuring that the output of each task is stored or shared appropriately within the Crew, you can create a cohesive memory system. This shared memory can be accessed by different agents in the Crew as they collaborate on complex tasks, fostering efficiency and knowledge sharing.

If you require further assistance on implementing memory management within your Crew using CrewAI, please provide us with more details on your specific requirements. We are here to offer tailored guidance to help you successfully establish and launch your Crew with an efficient memory management system.

For additional information and detailed instructions on creating a Crew and initiating tasks, you can refer to our CrewAI Documentation at https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/.

We are confident that with the right approach to memory management, your Crew will excel in handling tasks and processes seamlessly. Should you have any more questions or need further assistance, feel free to reach out to us anytime.

Best regards,
The CrewAI Support Team